In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


#Data prepocess with Pymol

In [ ]:
from IPython.utils import io
import tqdm.notebook
import os
"""The PyMOL installation is done inside two nested context managers. This approach
was inspired by Dr. Christopher Schlick's (of the Phenix group at
Lawrence Berkeley National Laboratory) method for installing cctbx
in a Colab Notebook. He presented his work on September 1, 2021 at the IUCr
Crystallographic Computing School. I adapted Chris's approach here. It replaces my first approach
that requires seven steps. My approach was presentated at the SciPy2021 conference
in July 2021 and published in the
[proceedings](http://conference.scipy.org/proceedings/scipy2021/blaine_mooers.html).
The new approach is easier for beginners to use. The old approach is easier to debug
and could be used as a back-up approach.

Thank you to Professor David Oppenheimer of the University of Florida for suggesting the use mamba and of Open Source PyMOL.
"""
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:

        !pip install -q condacolab
        import condacolab
        condacolab.install()
        pbar.update(10)

        import sys
        sys.path.append('/usr/local/lib/python3.7/site-packages/')
        pbar.update(20)

        # Install PyMOL
        %shell mamba install pymol-open-source --yes

        pbar.update(100)


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
from pymol import cmd
from IPython.display import Image

In [ ]:
# get protein ID
protein_file = open('/content/drive/MyDrive/pymol/pdb_ids', 'r')
protein_lines = protein_file.readlines()
id_list = []
for line in protein_lines:
  id_list.append(line.strip())


In [ ]:
len(id_list)

309

## generate pdf for only protein; and pockets position

In [ ]:
## for list[:-1]
%%capture cap
for id in id_list[:50]:
  print(str(id))
  #refresh
  cmd.delete("all")

  #load in PDB structure
  cmd.fetch(id)

  #remove solvent molecules
  cmd.remove("solvent")

  #select ligand
  cmd.select("ligand", "HETATM")
  cmd.color("blue", "ligand")

  #select residues around ligand
  cmd.select("pocket", "ligand around 5")
  cmd.color("green", "pocket")

  cmd.iterate("pocket and name CA", "print(chain, resi)")
  cmd.remove("ligand")

  # cmd.png("test.png")
  # Image(filename = "test.png", unconfined=True)

  # cmd.save(id + '.pdb', id)
f = open("/content/drive/MyDrive/pymol/temp1-50.txt", "w") 
print(cap, file=f)
f.close()


In [ ]:
%%capture cap
#refresh
cmd.delete("all")

#load in PDB structure
cmd.fetch('8EYJ')

#remove solvent molecules
cmd.remove("solvent")

#select ligand
cmd.select("ligand", "HETATM")
cmd.color("blue", "ligand")

#select residues around ligand
cmd.select("pocket", "ligand around 5")
cmd.color("green", "pocket")

cmd.iterate("pocket and name CA", "print(chain, resi)")
cmd.remove("ligand")

# cmd.png("test.png")
# Image(filename = "test.png", unconfined=True)

# cmd.save(id + '.pdb', id)
f = open("/content/drive/MyDrive/pymol/temp_last.txt", "w") 
print(cap, file=f)
f.close()


## merge positiob data to one file

In [ ]:
f = open("/content/drive/MyDrive/pymol/temp1-50.txt", "r")
output = open('/content/drive/MyDrive/pymol/resi1-50.txt', 'w')
lines = f.readlines()
resilist =[]
for line in lines:
  line = str(line.strip())
  if len(line) is not 0:
    if line[1] is not ' ':
      if len(resilist) == 0:
        header = line
        # print(header)
        # print(1)
      else:
        
        resultstr = ' '.join(resilist)
        content = header + '\t' + resultstr
        # print(content)
        print(content,file=output)
        resilist =[]
        header = line

    else:
      templist = line.split(' ')
      resi = ':'.join(templist)
      resilist.append(resi)
output.close()


In [ ]:
f = open("/content/drive/MyDrive/pymol/temp_last.txt", "r")
# output = open('/content/drive/MyDrive/pymol/resi1-50.txt', 'w')
lines = f.readlines()
resilist =[]
for line in lines:
  line = str(line.strip())
  templist = line.split(' ')
  resi = ':'.join(templist)
  resilist.append(resi)
resultstr = ' '.join(resilist)
print(resultstr)
output.close()


A:27 A:41 A:49 A:140 A:141 A:142 A:143 A:144 A:145 A:164 A:165 A:166 A:167 A:187 A:188 A:189 A:190 A:192 B:27 B:41 B:49 B:140 B:141 B:142 B:143 B:144 B:145 B:164 B:165 B:166 B:167 B:187 B:188 B:189 B:192 


## move pdb files to one folder

In [ ]:
import os, shutil

In [ ]:
path = '/content'
filelist = os.listdir(path)
pdbroot = '/content/drive/MyDrive/pymol/pdb'
cifroot = '/content/drive/MyDrive/pymol/cif'
for filename in filelist:
  temp = filename.split('.')
  if len(temp) == 2:
    if temp[1] == 'pdb':
      oldpath = os.path.join(path,filename)
      newpath = os.path.join(pdbroot,filename)
      shutil.move(oldpath,newpath)
    if temp[1] == 'cif':
      oldpath = os.path.join(path,filename)
      newpath = os.path.join(cifroot,filename)
      shutil.move(oldpath,newpath)

## generate sdf/pdb file for ligand

In [ ]:
import random
for id in id_list:
  #refresh
  cmd.delete("all")

  #load in PDB structure
  cmd.fetch(id)

  #remove solvent molecules
  cmd.remove("solvent")

  #select ligand
  cmd.select("ligand", "HETATM")
  cmd.color("blue", "ligand")


  cmd.remove("!ligand")
  # cmd.translate(vector = [random.uniform(-25, 25),random.uniform(-25, 25),random.uniform(-25, 25)], selection = "ligand")
  # cmd.rotate(axis = "x", angle = random.uniform(-180, 180), selection = "ligand")
  # cmd.rotate(axis = "y", angle = random.uniform(-180, 180), selection = "ligand")
  # cmd.rotate(axis = "z", angle = random.uniform(-180, 180), selection = "ligand")  

  cmd.save(id + '_ligand.sdf', id)


In [ ]:
id_list[2]

'5ST2'

In [ ]:
import os, shutil

In [ ]:
path = '/content'
filelist = os.listdir(path)
pdbroot = '/content/drive/MyDrive/pymol/ligand_sdf'
# cifroot = '/content/drive/MyDrive/pymol/cif'
for filename in filelist:
  temp = filename.split('.')
  if len(temp) == 2:
    if temp[1] == 'sdf':
      oldpath = os.path.join(path,filename)
      newpath = os.path.join(pdbroot,filename)
      shutil.move(oldpath,newpath)
    # if temp[1] == 'cif':
    #   oldpath = os.path.join(path,filename)
    #   newpath = os.path.join(cifroot,filename)
    #   shutil.move(oldpath,newpath)

In [ ]:
len(os.listdir('/content/drive/MyDrive/pymol/ligand_sdf'))

309

## test on one protein

In [ ]:
import random
#refresh
cmd.delete("all")

#load in PDB structure
cmd.fetch("8EYJ")

#remove solvent molecules
cmd.remove("solvent")

#select ligand
cmd.select("ligand", "HETATM")
cmd.color("blue", "ligand")

#select residues around ligand
# cmd.select("pocket", "ligand around 10")
# cmd.color("green", "pocket")

cmd.remove("!ligand")
cmd.translate(vector = [random.uniform(-25, 25),random.uniform(-25, 25),random.uniform(-25, 25)], selection = "ligand")
cmd.rotate(axis = "x", angle = random.uniform(-180, 180), selection = "ligand")
cmd.rotate(axis = "y", angle = random.uniform(-180, 180), selection = "ligand")
cmd.rotate(axis = "z", angle = random.uniform(-180, 180), selection = "ligand")  


# cmd.iterate("pocket and name CA", "print(chain, resi)")
# cmd.remove("ligand")

cmd.png("test.png")
Image(filename = "test.png", unconfined=True)

# cmd.save(x + '.pdb', x)
# cmd.delete(x)
cmd.save("8EYJ_ligand" + '.sdf', "8EYJ")
# cmd.delete("5st0")
# f = open("/content/drive/MyDrive/pymol/last.txt", "w") 
# print(cap, file=f)
# f.close()